This is another piece of one-time code. Once it was run, we don't want to mess with it again. I had to work through this several times as we worked out exactly how Brandon Serna's new usgs data tools package worked with the DOI tool. I'm keeping this around as a record of what we did to get the DOIs minted and on board the items. From here there will be other code to update or keep those DOIs in play.

In [14]:
import requests
import pysb
import time
from IPython.display import display
import getpass

from usgs_datatools import doi
from usgs_datatools.doi import add_primary_doi_manager

In [2]:
_rangeMapRoot = "5951527de4b062508e3b1e79"

In [3]:
sb = pysb.SbSession()
username = input("Username: ")
sb.loginc(str(username))

Username: sbristol@usgs.gov
········


In [4]:
username = 'sbristol@usgs.gov'
password = getpass.getpass('USGS AD Password: ')

USGS AD Password: ········


In [5]:
DoiSession = doi.DoiSession(env='production')

In [6]:
DoiSession.doi_authenticate(username, password)

In [24]:
rangeMapDocs = []
rangeMapSearchResults = sb.find_items("parentId="+_rangeMapRoot+"&fields=title,body,contacts,identifiers&max=100")
while rangeMapSearchResults is not None:
    for rangeItem in rangeMapSearchResults["items"]:
        thisRangeMapDoc = {}
        thisRangeMapDoc["id"] = rangeItem["id"]
        thisRangeMapDoc["identifiers"] = rangeItem["identifiers"]
        thisRangeMapDoc["title"] = rangeItem["title"]
        rangeMapDocs.append(thisRangeMapDoc)
    rangeMapSearchResults = sb.next(rangeMapSearchResults)

In [23]:
count = 0
for rangeItem in rangeMapDocs:
    doiRegistration = {}
    doiRegistration["status"] = "reserved"
    doiRegistration["resourceURL"] = "https://www.sciencebase.gov/catalog/item/"+rangeItem["id"]
    doiRegistration["title"] = rangeItem["title"]
    doiRegistration["abstractTypeDesc"] = rangeItem["body"]
    doiRegistration["pubYear"] = "2017"
    doiRegistration["resourceType"] = "Dataset"
    authorNames = []
    for index,contact in enumerate(rangeItem["contacts"]):
        if contact["type"] == "reviewer":
            lastAuthor = contact
        else:
            if contact["name"] not in authorNames:
                doiRegistration["authors["+str(index - 1)+"].position"] = str(index - 1)
                doiRegistration["authors["+str(index - 1)+"].orcId"] = ""
                doiRegistration["authors["+str(index - 1)+"].authorName"] = contact["name"].split(" ")[-1]+", "+contact["name"].split(" ")[0]
                lastAuthorPosition = index - 1
                authorNames.append(contact["name"])
    if lastAuthor is not None and lastAuthor["name"] not in authorNames:
        doiRegistration["authors["+str(lastAuthorPosition + 1)+"].position"] = str(lastAuthorPosition + 1)
        doiRegistration["authors["+str(lastAuthorPosition + 1)+"].orcId"] = ""
        doiRegistration["authors["+str(lastAuthorPosition + 1)+"].authorName"] = lastAuthor["name"].split(" ")[-1]+", "+lastAuthor["name"].split(" ")[0]
    
    add_primary_doi_manager(doiRegistration, 'sbristol@usgs.gov')
    
    updateRangeItem = {}
    updateRangeItem["id"] = rangeItem["id"]
    updateRangeItem["identifiers"] = rangeItem["identifiers"]

    try:
        count = count + 1
        theDOI = DoiSession.doi_create(doiRegistration)
        updateRangeItem["identifiers"].append({"type":"doi","scheme":"https://www.sciencebase.gov/vocab/term/528e9a2ce4b05d51c7038afe","key":theDOI})
        sb.update_item(updateRangeItem)
        print (rangeItem["id"], theDOI, count)
    except Exception as e:
        existingDOI = doiDataDump.loc[doiDataDump["url"] == doiRegistration["resourceURL"]]["doi"].to_string(index=False)
        if existingDOI is not None:
            updateRangeItem["identifiers"].append({"type":"doi","scheme":"https://www.sciencebase.gov/vocab/term/528e9a2ce4b05d51c7038afe","key":existingDOI})
            sb.update_item(updateRangeItem)
            print (rangeItem["id"], existingDOI, count, "UPDATED ScienceBase WITH PREVIOUSLY MINTED DOI")
        else:
            print (rangeItem["id"], e)
            display (doiRegistration)


59f5e1b4e4b063d5d307da77 doi:10.5066/F7RR1XDZ 1 UPDATED ScienceBase WITH PREVIOUSLY MINTED DOI
59f5ec28e4b063d5d307e4a9 doi:10.5066/F7RR1XDZ 2 UPDATED ScienceBase WITH PREVIOUSLY MINTED DOI
59f5e235e4b063d5d307dd3f doi:10.5066/F7RR1XDZ 3 UPDATED ScienceBase WITH PREVIOUSLY MINTED DOI
Created DOI: doi:10.5066/F7H70DZ6
59f5e285e4b063d5d307df19 doi:10.5066/F7H70DZ6 4
Created DOI: doi:10.5066/F7CF9P7B
59f5ec6de4b063d5d307e6fb doi:10.5066/F7CF9P7B 5
Created DOI: doi:10.5066/F77P8XJ2
59f5ebf7e4b063d5d307e2f7 doi:10.5066/F77P8XJ2 6
Created DOI: doi:10.5066/F7416W6X
59f5e270e4b063d5d307dea1 doi:10.5066/F7416W6X 7
Created DOI: doi:10.5066/F70864GK
59f5e264e4b063d5d307de5d doi:10.5066/F70864GK 8
Created DOI: doi:10.5066/F7VH5N09
59f5e21ae4b063d5d307dca5 doi:10.5066/F7VH5N09 9
Created DOI: doi:10.5066/F7QR4W8K
59f5e1e9e4b063d5d307db8d doi:10.5066/F7QR4W8K 10
Created DOI: doi:10.5066/F7M044JJ
59f5e1ede4b063d5d307db9b doi:10.5066/F7M044JJ 11
Created DOI: doi:10.5066/F7G73CV8
59f5e1ede4b063d5d307db9